In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, Lasso, LinearRegression,RidgeCV
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

plt.style.use('ggplot')
%matplotlib inline

In [50]:
df=pd.read_csv('lasso.csv',index_col=0)

In [51]:
df.head()

,LotFrontage,LotArea,MasVnrArea,TotalBsmtSF,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,...,Functional,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MoSold_Autumn,MoSold_Spring,MoSold_Winter
Id,,,,,,,,,,,,,,,,,,,,,
1,65.0,8450,196.0,856,3,1,8,0,2,548,...,8,1,3,4,4,1,1,0,0,1
2,80.0,9600,0.0,1262,3,1,6,1,2,460,...,8,4,3,4,4,1,1,0,1,0
3,68.0,11250,162.0,920,3,1,6,1,2,608,...,8,4,3,4,4,1,1,1,0,0
4,60.0,9550,0.0,756,3,1,7,1,3,642,...,8,5,2,4,4,1,1,0,0,1
5,84.0,14260,350.0,1145,4,1,9,1,3,836,...,8,4,3,4,4,1,1,0,0,1


In [52]:
SalePrice=df.SalePrice
ridge_df=df.drop('SalePrice',axis=1)

### Standardize numeric columns

In [41]:
#check maximum value of each feature to decide which columns we want to normalize
list(map(lambda x:print(x,max(ridge_df[x])),ridge_df))

LotFrontage 313.0
LotArea 215245
MasVnrArea 1600.0
TotalBsmtSF 3206
BedroomAbvGr 8
KitchenAbvGr 3
TotRmsAbvGrd 14
Fireplaces 3
GarageCars 4
GarageArea 1390
WoodDeckSF 857
OpenPorchSF 547
EnclosedPorch 552
ScreenPorch 480
BsmtScore 5.997715851987209
TotalSF 4476
TotalBath 6.0
MSSubClass_60 1
MSSubClass_75 1
MSSubClass_90 1
MSSubClass_120 1
MSSubClass_160 1
MSSubClass_190 1
MSZoning_FV 1
MSZoning_RH 1
MSZoning_RM 1
Street_Grvl 1
Alley_Grvl 1
LandContour_Bnk 1
LandContour_HLS 1
LotConfig_Corner 1
LotConfig_CulDSac 1
LotConfig_FR2 1
Neighborhood_BrDale 1
Neighborhood_BrkSide 1
Neighborhood_CollgCr 1
Neighborhood_Crawfor 1
Neighborhood_Edwards 1
Neighborhood_Gilbert 1
Neighborhood_Mitchel 1
Neighborhood_NPkVill 1
Neighborhood_NWAmes 1
Neighborhood_NoRidge 1
Neighborhood_NridgHt 1
Neighborhood_Sawyer 1
Neighborhood_StoneBr 1
Neighborhood_Veenker 1
BldgType_Duplex 1
BldgType_Twnhs 1
BldgType_TwnhsE 1
HouseStyle_1.5Fin 1
HouseStyle_2Story 1
HouseStyle_SFoyer 1
HouseStyle_SLvl 1
RoofStyle_Hip 1

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [53]:
'''The function to standardize the columns seperately. 
df is the dataframe needs the standardization.  
collist is the list of columns which need to be standardized'''


def standize_df(df,collist):
    for col in collist:
        stan_df=df[[col]]
        min_max_scaler = preprocessing.MinMaxScaler()
        standardized=min_max_scaler.fit_transform(stan_df)
        stan2_df=pd.DataFrame(standardized,columns=stan_df.columns,index=stan_df.index)
        df[[col]]=stan2_df
    return df

# Standarize column with maximum values larger than 100 
collist=['LotFrontage','LotArea','MasVnrArea','TotalBsmtSF','GarageArea','WoodDeckSF','OpenPorchSF',
         'EnclosedPorch','ScreenPorch','TotalSF','YearsAgoBuilt']
ridge_df=standize_df(ridge_df,collist)
ridge_df.head()

,LotFrontage,LotArea,MasVnrArea,TotalBsmtSF,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageCars,GarageArea,...,Functional,FireplaceQu,GarageFinish,GarageQual,GarageCond,PoolQC,Fence,MoSold_Autumn,MoSold_Spring,MoSold_Winter
Id,,,,,,,,,,,,,,,,,,,,,
1,0.150685,0.033420,0.12250,0.266999,3,1,8,0,2,0.394245,...,8,1,3,4,4,1,1,0,0,1
2,0.202055,0.038795,0.00000,0.393637,3,1,6,1,2,0.330935,...,8,4,3,4,4,1,1,0,1,0
3,0.160959,0.046507,0.10125,0.286962,3,1,6,1,2,0.437410,...,8,4,3,4,4,1,1,1,0,0
4,0.133562,0.038561,0.00000,0.235808,3,1,7,1,3,0.461871,...,8,5,2,4,4,1,1,0,0,1
5,0.215753,0.060576,0.21875,0.357143,4,1,9,1,3,0.601439,...,8,4,3,4,4,1,1,0,0,1


In [54]:
# Split the Ames_df into a 80% train-set and a 20% test set.
np.random.seed(19)
testIdxes = np.random.choice(range(1458), size= 292, replace=False)
trainIdxes = list(set(range(1458))-set(testIdxes))
trainX = ridge_df.iloc[trainIdxes]
testX  = ridge_df.iloc[testIdxes]
trainY = SalePrice.iloc[trainIdxes]
testY  = SalePrice.iloc[testIdxes]


#get best lambda
alphaRange = np.linspace(1e-3,200,40) # Use np.linspace to generate 40 alpha
ridge_cv = RidgeCV(alphas = alphaRange,cv=10) #cross validation to get best lambda
ridge_cv.fit(trainX, trainY)
best_alpha=ridge_cv.alpha_
best_alpha=round(best_alpha,3)
best_alpha

0.001

In [55]:

#Train ridge
ridge=Ridge()
ridge.set_params(alpha=best_alpha)
ridge.fit(trainX,trainY)
ridge_intercept = ridge.intercept_
ridge_coef = pd.Series(ridge.coef_,index=ridge_df.columns)
rtrain_score=ridge.score(trainX,trainY)
rtest_score= ridge.score(testX,testY)

In [56]:
print ('intercept \t',ridge_intercept)
print ('train_score(R2)\t',rtrain_score)
print ('test_score(R2) \t',rtest_score)

intercept 	 -137885.7376460612
train_score(R2)	 0.9218589149670022
test_score(R2) 	 0.8806820407915643


In [57]:
ridge_predict = ridge.predict(testX)
RMSE=np.sqrt(mean_squared_error(testY,ridge_predict))
RMSE

23536.07265423612

In [58]:
#Sorted the coefs, the larger the number is , the larger impact it has to the whole model
#showing the top 60 features
sorted_coef=abs(ridge_coef).sort_values(ascending=False)
sorted_coef[1:60]

LotArea                  115058.536761
RoofMatl_WdShngl          67633.533859
TotalBsmtSF               60318.706909
MasVnrArea                52840.232506
Neighborhood_StoneBr      48379.981663
Street_Grvl               42457.978175
Heating_OthW              38810.217752
SaleType_CWD              36170.889177
YearsAgoBuilt             34844.304021
GarageType_No             32052.647123
Neighborhood_NridgHt      28918.720454
Neighborhood_NoRidge      26494.194315
Neighborhood_NPkVill      26325.540220
Exterior_ImStucc          25152.892439
PoolQC                    23776.367338
Neighborhood_Crawfor      21734.487852
BldgType_Twnhs            21726.327513
MSSubClass_75             20350.596906
SaleType_ConLI            20050.784492
ScreenPorch               16386.716076
SaleCondition_Partial     16321.673175
Condition_RRAe            16237.348281
BldgType_TwnhsE           14678.707326
KitchenAbvGr              14632.981105
RoofMatl_WdShake          14518.149050
HouseStyle_SFoyer        

In [59]:
PredictionsTest=ridge.predict(testX)
RMSLEvalue = np.sqrt(np.mean(np.power(np.log1p(testY)-np.log1p(PredictionsTest), 2)))
RMSLEvalue

0.15640697915092874